In [ ]:
!pip install datasets rouge

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 5.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.2/401.2 kB 9.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
import nltk
from nltk.tokenize import sent_tokenize
# Ensure that the necessary NLTK resources are downloaded
nltk.download('punkt')

def lead_5_summarization(text):
    """
    Summarize the text by extracting the first three sentences.
    Args:
    text (str): The text to be summarized.

    Returns:
    str: The summary of the text.
    """
    # Tokenize the text into sentences
    sentences = sent_tokenize(text)

    # Extract the first three sentences or fewer if the text is shorter
    summary = ' '.join(sentences[:5])

    return summary

# Example text
example_text = """
COVID-19 is an infectious disease caused by a newly discovered coronavirus. Most people infected with the COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment. Older people, and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and cancer are more likely to develop serious illness. The best way to prevent and slow down transmission is to be well informed about the COVID-19 virus, the disease it causes and how it spreads. Protect yourself and others from infection by washing your hands or using an alcohol based rub frequently and not touching your face.
"""

# Generate summary
summary = lead_5_summarization(example_text)
print("Summary:")
print(summary)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


Summary:

COVID-19 is an infectious disease caused by a newly discovered coronavirus. Most people infected with the COVID-19 virus will experience mild to moderate respiratory illness and recover without requiring special treatment. Older people, and those with underlying medical problems like cardiovascular disease, diabetes, chronic respiratory disease, and cancer are more likely to develop serious illness. The best way to prevent and slow down transmission is to be well informed about the COVID-19 virus, the disease it causes and how it spreads. Protect yourself and others from infection by washing your hands or using an alcohol based rub frequently and not touching your face.


In [ ]:
from datasets import load_dataset

raw_datasets = load_dataset("allenai/cord19", "fulltext", "abstract", trust_remote_code=True)

Generating train split:   0%|          | 0/368618 [00:00<?, ? examples/s]

In [ ]:
import numpy as np
def select_datapints(dataset):
    dataset['select'] = (len(dataset['fulltext']) > 0) and (len(dataset['abstract']) > 0)
    return dataset

raw_datasets['train'] = raw_datasets['train'].map(select_datapints)
indices_required = np.where(np.array(raw_datasets['train']['select']) == True)[0]
raw_datasets['train'] = raw_datasets['train'].select(indices_required)
raw_datasets


Map:   0%|          | 0/368618 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['cord_uid', 'sha', 'source_x', 'title', 'doi', 'abstract', 'publish_time', 'authors', 'journal', 'url', 'fulltext', 'select'],
        num_rows: 105097
    })
})

In [ ]:
raw_datasets = raw_datasets.remove_columns(['cord_uid', 'sha', 'source_x', 'title', 'doi', 'publish_time', 'authors', 'journal', 'url'])

import gc
gc.collect()

from datasets import DatasetDict
train_testvalid = raw_datasets['train'].train_test_split(test_size=0.2)

# Further split the test+validation into test and validation sets evenly
test_valid = train_testvalid['test'].train_test_split(test_size=0.5)

gc.collect()

test_valid = test_valid['test']

In [ ]:
import rouge

def get_rouge_scores(document):

    generated_text = lead_5_summarization(document['fulltext'])
    sentence_rouge_score = rouge.Rouge().get_scores(generated_text,
                                                    document['abstract'])[0]
    for rog in sentence_rouge_score:
        document[rog] = sentence_rouge_score[rog]['f']


    return document



In [ ]:
test = test_valid.map(get_rouge_scores, num_proc=8)

np.mean(test['rouge-1']), np.mean(test['rouge-2']), np.mean(test['rouge-l'])

In [ ]:
!pip install gensim

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
df = pd.read_csv("data_val.csv")

In [ ]:
text_data = ''
for i in range(2000):
  s = df["fulltext"][i]
  text_data = text_data + s

In [ ]:
import re

In [ ]:
def clean_text(text):
    # Remove punctuation
    text = re.sub(r'[^\w\s]', '', text)

    # Remove numbers
    text = re.sub(r'\d+', '', text)

    # Remove empty brackets
    text = re.sub(r'\[\s*\]', '', text)

    return text

text_data = clean_text(text_data)

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, sent_tokenize
import nltk
import os



# Tokenize the text data into sentences
sentences = [word_tokenize(sentence.lower()) for sentence in sent_tokenize(text_data)]

# Train Word2Vec model
model = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1, workers=4)

# Save the trained model
model.save('word2vec_model.model')

# Load the trained model
# model = Word2Vec.load('word2vec_model.model')

# Access word vectors
word_vectors = model.wv

# Get the vector for a specific word
vector = word_vectors['text']

# Print the vector representation of the word 'text'
print("Vector for 'text':", vector)


Vector for 'text': [ 0.00146238  0.00317007  0.00704033 -0.00183726 -0.00438365 -0.00805439
 -0.00425735 -0.00728582 -0.00669291  0.0063109  -0.00975406 -0.00083097
  0.00668586  0.00298326 -0.00085011 -0.00600237 -0.00349684 -0.00171481
 -0.00930345 -0.00703112 -0.00981087  0.00496471  0.00680523 -0.00055621
  0.00339235  0.00847581  0.00454307  0.00706092  0.00838828  0.0032961
 -0.00259828  0.00021587  0.0074653  -0.00077626  0.0036094  -0.00159053
 -0.00543391 -0.00597013  0.00654499 -0.0070722  -0.00157108  0.00932182
 -0.00437296 -0.00251411  0.00444982 -0.00727291 -0.00468064 -0.00269294
 -0.00110516 -0.0043072   0.00442158 -0.00279271  0.00184014 -0.0030181
 -0.00875286  0.00990137 -0.0092214  -0.00517091  0.00422765 -0.00888695
  0.0075977  -0.00513065 -0.00021808 -0.00743285  0.00607086 -0.00449997
  0.00350753  0.00683662 -0.00725652 -0.00317194  0.00263299 -0.00014413
  0.00997168 -0.00484205  0.00919367  0.00589131 -0.00343034  0.00499036
  0.007813    0.00949158  0.005580

In [ ]:
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize, sent_tokenize
import numpy as np
import networkx as nx

def sentence_vector(sentence, model):
    """
    Generate a sentence vector by averaging Word2Vec vectors of constituent words.
    """
    words = word_tokenize(sentence)
    word_vectors = [model.wv[word] for word in words if word in model.wv.key_to_index]
    if len(word_vectors) == 0:
        return None
    return np.mean(word_vectors, axis=0)

def generate_summary(text, model, num_sentences=5):
    """
    Perform text summarization using TextRank algorithm with Word2Vec embeddings.
    """
    # Tokenize text into sentences
    sentences = sent_tokenize(text)

    # Generate sentence vectors
    sentence_vectors = [sentence_vector(sentence, model) for sentence in sentences]

    # Calculate cosine similarity between sentence vectors
    similarity_matrix = np.zeros((len(sentences), len(sentences)))
    for i in range(len(sentences)):
        for j in range(len(sentences)):
            if i != j:
                similarity_matrix[i][j] = cosine_similarity(sentence_vectors[i].reshape(1, -1), sentence_vectors[j].reshape(1, -1))[0, 0]

    # Build graph from similarity matrix
    G = nx.from_numpy_array(similarity_matrix)

    # Apply PageRank algorithm
    scores = nx.pagerank(G)

    # Sort sentences by importance score
    ranked_sentences = sorted(((scores[i], sentence) for i, sentence in enumerate(sentences)), reverse=True)

    # Select top-ranked sentences for summary
    summary_sentences = [sentence for score, sentence in ranked_sentences[:num_sentences]]

    return ' '.join(summary_sentences)


# Load the trained Word2Vec model
model = Word2Vec.load('word2vec_model.model')

In [ ]:
import sys
sys.setrecursionlimit(10**9)  # Set a higher recursion limit


In [ ]:
from rouge import Rouge

x = []
'''
j = clean_text(df['fulltext'][4000])
generated_summary = generate_summary(j,model)
rouge = Rouge()

scores = rouge.get_scores(generated_summary,df['abstract'][0])
print(scores)
# Reference summary
'''
rouge = Rouge()
for i in range(4000,4010,1):
# Generate summary using the trained Word2Vec model
  df['fulltext'][i] = clean_text(df['fulltext'][i])
  generated_summary = generate_summary(df['fulltext'][i],model)


  scores = rouge.get_scores(generated_summary, df['abstract'][i])
  print(scores)
  x.append(scores)





In [ ]:
x

[[{'rouge-1': {'r': 0.8488372093023255,
    'p': 0.05505279034690799,
    'f': 0.1033994322838238},
   'rouge-2': {'r': 0.4326241134751773,
    'p': 0.01492354740061162,
    'f': 0.02885183806882663},
   'rouge-l': {'r': 0.6686046511627907,
    'p': 0.043363499245852186,
    'f': 0.08144475806286064}}],
 [{'rouge-1': {'r': 0.6794871794871795,
    'p': 0.12709832134292565,
    'f': 0.21414141148650137},
   'rouge-2': {'r': 0.2777777777777778,
    'p': 0.031660983925962005,
    'f': 0.05684302396101696},
   'rouge-l': {'r': 0.5, 'p': 0.09352517985611511, 'f': 0.15757575492084486}}],
 [{'rouge-1': {'r': 0.573134328358209,
    'p': 0.3522935779816514,
    'f': 0.436363631648373},
   'rouge-2': {'r': 0.42015209125475284,
    'p': 0.20751173708920187,
    'f': 0.2778126919912092},
   'rouge-l': {'r': 0.5074626865671642,
    'p': 0.3119266055045872,
    'f': 0.38636363164837295}}],
 [{'rouge-1': {'r': 0.5944055944055944,
    'p': 0.07112970711297072,
    'f': 0.1270553045184344},
   'rouge-2'